In [1]:
import pandas as pd

In [2]:
# read csvs as data frames
# cast as string to avoid float/NaN errors

productions = pd.read_csv('in/productions.csv', dtype=object)
productions_images = pd.read_csv('in/productions_images.csv', dtype=object)
performances = pd.read_csv('in/performances.csv', dtype=object)
imagesonline = pd.read_csv('in/imagesonline.csv', dtype=object)

In [3]:
# mild clean-up/renaming
productions = productions.rename(columns = {'label':'label_eng', 'id':'production_id','full_date':'dates'})

performances = performances.drop('data_id',1).drop('code',1)
performances = performances.rename(columns = {'id':'performance_id'})
imagesonline = imagesonline.rename(columns = {'id':'image_id'})

In [4]:
productions.head()

,production_id,seq_chron,label_eng,dates,place,performance_num
0,1,001,November 1964,1964/11/n.d.-1964/11/n.d.,not recorded,NaN
1,2,002,February 1968,1968/02/25-1968/03/10,National Theatre of Japan,006
2,3,003,October 1968,1968/10/27-1968/11/10,National Theatre of Japan,008
3,4,004,May 1969,1969/05/11-1969/05/25,National Theatre of Japan,010
4,5,005,September 1969,1969/09/14-1969/09/21,National Theatre of Japan,011


In [5]:
# clean out non-online images
imagesonline = imagesonline[['image_id']]
print 'online: ' + str(len(imagesonline.index)) + ' / in join: ' + str(len(productions_images.index)) + '\n'
productions_images = pd.merge(productions_images, imagesonline, on='image_id', how='inner')
print 'online in join: ' + str(len(productions_images.index))

online: 14636 / in join: 18771

online in join: 13494


In [6]:
productions_images.head()

,production_id,image_id
0,86,8281
1,86,8282
2,86,8283
3,86,8284
4,86,8285


In [7]:
# multiValDataFrame takes a dataframe df, column name df_index and column name multival_key
# it merges records with the same df_index and updates multival_index to an array of all the multival_index values.
# E.g for a data frame plays with multiple character_id values per play_id, returns a data frame with one record 
# per play_id, and one array of character_ids per play.

def multiValDataFrame(df, df_index, multival_key):
    new_df = pd.DataFrame({})
    cols = df.columns
    id_list = df[df_index].unique()
    for i in id_list:
        temp_row = df.loc[df[df_index] == i ]
        vals = []
        for col in cols:
            vals.append(temp_row[:1][col].values[0])

        temp_dict = dict(zip(cols, vals))
        multi_id_list = list(temp_row[multival_key])
    
        temp_dict[multival_key] = multi_id_list
    
        new_df = new_df.append(temp_dict, ignore_index=True)
    return new_df

In [8]:
# merge and multival images
productions = pd.merge(productions, productions_images, on='production_id', how='left')
productions = multiValDataFrame(productions, 'production_id', 'image_id')

In [9]:
# merge and multival images
productions = pd.merge(productions, performances, on='production_id', how='left')
productions = multiValDataFrame(productions, 'production_id', 'performance_id')

In [10]:
# mild clean-up / re-ordering
productions = productions.rename(columns = {'production_id':'id'})
productions = productions[['id','dates','place','label_eng','seq_chron','image_id','performance_id','play_id']]

In [11]:
productions.head()

,id,dates,place,label_eng,seq_chron,image_id,performance_id,play_id
0,1,1964/11/n.d.-1964/11/n.d.,not recorded,November 1964,001,[nan],"[2, 3, 4, 5, 6, 7, 8]",86
1,2,1968/02/25-1968/03/10,National Theatre of Japan,February 1968,002,[nan],"[12, 13, 14, 15, 16, 17]",83
2,3,1968/10/27-1968/11/10,National Theatre of Japan,October 1968,003,[nan],"[9, 10, 11]",30
3,4,1969/05/11-1969/05/25,National Theatre of Japan,May 1969,004,[nan],"[18, 19, 20]",72
4,5,1969/09/14-1969/09/21,National Theatre of Japan,September 1969,005,[nan],"[21, 22, 23]",90


In [12]:
# export to csv
productions.to_csv('productions_exp.csv', encoding='utf8', index=False)

In [13]:
# export to json
productions.to_json('productions_exp.json', orient='records', force_ascii=False)